In [14]:
#PigLab2NoteBook

! pip install pandas
! pip install openpyxl
!pip install rdflib
import pandas as pd
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit
from pyspark.sql.functions import when, col, lit

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


### Step 01: Define the file paths and Load the data into Pandas dataframes and Clean

In [15]:
# Define the file paths
files = ['data/PigData/Lab2/DECIDE INFLUENZA 20-22.xlsx']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    df = pd.read_excel(file, engine='openpyxl')
    dfs.append(df)


df = dfs[0]


In [16]:
df

,Nº Informe,Fecha recepción,Tipo muestra,Referencia,PCR-Influenza
0,V37547,2020-09-24,Fluido oral (PORCINA),Lleida,Negativo
1,V37548,2020-09-24,Fluido oral (PORCINA),Lleida,Negativo
2,V37549,2020-09-24,Fluido oral (PORCINA),Lleida,Negativo
3,V37550,2020-09-24,Fluido oral (PORCINA),Lleida,Negativo
4,V37551,2020-09-24,Fluido oral (PORCINA),Lleida,Negativo
...,...,...,...,...,...
463,V46449,2022-12-15,Porción pulmón 2 ud (PORCINA),SAMANES-4 ES312490000027,Negativo
464,V46466,2022-12-15,Pulmón (PORCINA),MIR,Negativo
465,V47845,2022-12-23,Fluido oral (PORCINA),EL RASO,Negativo
466,V47846,2022-12-23,Fluido oral (PORCINA),EL RASO,Negativo


In [17]:
df = df.rename(columns={
    'Nº Informe': 'SampleNumber',
    'Fecha recepción': 'Date',
    'PCR-Influenza': 'Result',
    'Tipo muestra': 'SampleType',
    'Referencia': 'Province',
})

# Set DiagnosticTest to "PCR" Country to "Spain", Breed to "Meat", and LabReference to "1".
df['DiagnosticTest'] = 'PCR'
df['Pathogen'] = 'Swine influenza virus'
df['Country'] = 'Spain'
df['Breed'] = 'Meat'
df["PigStage"] = "Missing"
df["Lab_Reference"] = "2"
# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "Negativo": "0",
    "Positivo": "1",
}
df["Result"] = df["Result"].map(resultMapping).fillna("Missing")


SampleTypeMapping = {

    'Fluido oral (PORCINA)': 'Fluid',
    'Pulmón (PORCINA)': 'Tissue',
    'Torunda nasal (PORCINA)': 'Swab',
    'Vísceras: Pulmón, Riñón, Bazo, Hígado (PORCINA)': 'Tissue',
    'Torunda pool de 4 (PORCINA)': 'Swab',
    'Torunda nasal Pool de 3 (PORCINA)': 'Swab',
    'Suero (PORCINA)': 'Fluid',
    'Feto (PORCINA)': 'Tissue',
    'Torunda traqueal (PORCINA)': 'Swab',
    'Torunda (PORCINA)': 'Swab',
    'Porción pulmón (PORCINA)': 'Tissue',
    'Porción Pulmón 3 ud (PORCINA)': 'Tissue',
    'Lechón muerto (PORCINA)': 'Tissue',
    'Porción pulmón 2ud (PORCINA)': 'Tissue',
    'Pulmón 2ud + Líquido pericárdico 2ud (PORCINA)': 'Tissue',
    'Pulmón 3ud (PORCINA)': 'Tissue',
    'Torunda pulmonar (PORCINA)': 'Swab',
    'Pulmón+Líquido pericárdico 2ud (PORCINA)': 'Tissue',
    'Pulmón 2ud (PORCINA)': 'Tissue',
    'Lechón abortado (PORCINA)': 'Tissue',
    'Pulmón 2 ud (PORCINA)': 'Tissue',
    'Porción pulmón + Torunda líquido pericárdico (PORCINA)': 'Tissue',
    'Pulmón + corazón (PORCINA)': 'Tissue',
    'Porción pulmón 2 ud (PORCINA)': 'Tissue'
}
df["SampleType"] = df["SampleType"].map(SampleTypeMapping ).fillna("Missing")


ProvinceMapping = {
    'Lleida': 'Lleida',
    'Barcelona': 'Barcelona',
    'CERRO-LA ESTACION': 'Missing',
    '---': 'Missing',
    'GRANJA MATEU': 'Missing',
    'FONTANELLES': 'Missing',
    'PERE ROVIRA': 'Missing',
    'ENRIQUE VERGARA': 'Missing',
    'URGELL': 'Missing',
    'MAS POUS': 'Missing',
    'EL CODRIAL': 'Missing',
    'RAMON GIBERT 383FA': 'Missing',
    'GRANJA CAN BOSCH': 'Missing',
    'DAVIU': 'Missing',
    'EL CID': 'Missing',
    'SÁNCHEZ-JURADO': 'Missing',
    'GRANJA SENA': 'Missing',
    'AGROGANADERA MASE': 'Missing',
    'ARAMUNT': 'Missing',
    'PINSOS I RAMADERIA SOLÀ': 'Missing',
    'EST': 'Missing',
    'CUCURULLS': 'Missing',
    'SINTO': 'Missing',
    'POZO SECO': 'Missing',
    'PEÑALBA': 'Missing',
    'GRANJA CANTOSALES': 'Missing',
    'VELASCO': 'Missing',
    'BIDRIOKE': 'Missing',
    'LUMBIERRES': 'Missing',
    'GRANJA FRANQUIM': 'Missing',
    'FELBET RAMADERA': 'Missing',
    'MACHÓN': 'Missing',
    'SANTOMERA': 'Missing',
    'LAS MALEZAS': 'Missing',
    'GRANJA FELIU': 'Missing',
    'LA SIERRA': 'Missing',
    'ERA FELIU': 'Missing',
    'ROMERAL': 'Missing',
    'ES140510000227': 'Missing',
    'FAYÓN ABUELAS': 'Missing',
    'CANOTIER': 'Missing',
    'GRANJA HAYAS': 'Missing',
    'CABECERAN': 'Missing',
    'RAMON MIQUELO': 'Missing',
    'RAMON LUMBIERRES': 'Missing',
    'GRANJA LA RIERA': 'Missing',
    'BOSCH': 'Missing',
    'REGUÉ': 'Missing',
    'CAIROL-ESTISANER': 'Missing',
    'COSCOLLA': 'Missing',
    'MASO': 'Missing',
    'GRANJA RODELLAR': 'Missing',
    'MONCAYO': 'Missing',
    'ARBELADENA': 'Missing',
    '3830UL': 'Missing',
    'GRUPO ALTOPASO': 'Missing',
    'ALTOPASO': 'Missing',
    'TOMILLOS': 'Missing',
    'GRANGENIA': 'Missing',
    'PORCIN': 'Missing',
    'ESTISANER MESALLES': 'Missing',
    'MAIALS': 'Missing',
    'TABERNAS': 'Missing',
    'GRANJA DOS HERMANAS': 'Missing',
    'PI-GÓMEZ': 'Missing',
    '3600DG - CATARÓ': 'Missing',
    'CARRICONDO': 'Missing',
    'QUILES': 'Missing',
    'GRANJA ALPEDROCHE-CABANILLAS': 'Missing',
    'GRANJA EL GUINART': 'Missing',
    'GRANJA LOS PROPIOS': 'Missing',
    'CHUMBO': 'Missing',
    'MOLÍ ZANUY': 'Missing',
    'GASCÓN VICENTE': 'Missing',
    'SERRA': 'Missing',
    'ALBATERA': 'Missing',
    'HOSTAL DEL VENT': 'Missing',
    'LLOBET': 'Missing',
    'GALEON-MAQUEDA': 'Missing',
    'LLINAS': 'Missing',
    'GRANJA LOPEZ': 'Missing',
    'CEBO ANGEL VENTAJA': 'Missing',
    'VILAMALLA': 'Missing',
    'SANT BERNABÉ-ORIGEN MONREAL': 'Missing',
    'GRANJA MARÍA VAQUERA': 'Missing',
    'GANADOS LLOSIN': 'Missing',
    'MAS MARTÍ': 'Missing',
    'CEBADERO GALEÓN I': 'Missing',
    'LA CERCA EL ALTO': 'Missing',
    'LLORENS': 'Missing',
    'GRANJA SANCO': 'Missing',
    'LLORAC - 3930AC': 'Missing',
    'MORGÓ 001AP': 'Missing',
    "SOLER DE L'ESPINA": 'Missing',
    'FONESCO': 'Missing',
    'LA CERCA': 'Missing',
    'VILCHEZ': 'Missing',
    'CAN BAIXERES': 'Missing',
    'SUSAGNA - Bassa Nova': 'Missing',
    'SABATÉ ALGUAIRE': 'Missing'
}

df["Province"] = df["Province"].map(ProvinceMapping).fillna("Missing")

df['Province'] = df['Province'].fillna('Missing')

# Convert 'FECHA' column to datetime format
#df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')






In [18]:
df.head(50)

,SampleNumber,Date,SampleType,Province,Result,DiagnosticTest,Pathogen,Country,Breed,PigStage,Lab_Reference
0,V37547,2020-09-24,Fluid,Lleida,0,PCR,Swine influenza virus,Spain,Meat,Missing,2
1,V37548,2020-09-24,Fluid,Lleida,0,PCR,Swine influenza virus,Spain,Meat,Missing,2
2,V37549,2020-09-24,Fluid,Lleida,0,PCR,Swine influenza virus,Spain,Meat,Missing,2
3,V37550,2020-09-24,Fluid,Lleida,0,PCR,Swine influenza virus,Spain,Meat,Missing,2
4,V37551,2020-09-24,Fluid,Lleida,0,PCR,Swine influenza virus,Spain,Meat,Missing,2
5,V37552,2020-09-24,Fluid,Lleida,0,PCR,Swine influenza virus,Spain,Meat,Missing,2
6,V37818,2020-09-25,Fluid,Barcelona,0,PCR,Swine influenza virus,Spain,Meat,Missing,2
7,V37819,2020-09-25,Fluid,Barcelona,0,PCR,Swine influenza virus,Spain,Meat,Missing,2
8,V38583,2020-09-30,Fluid,Missing,0,PCR,Swine influenza virus,Spain,Meat,Missing,2
9,V38584,2020-09-30,Fluid,Missing,0,PCR,Swine influenza virus,Spain,Meat,Missing,2


In [19]:
# Save to desired path in CSV Format
df.to_csv('output/PigLab2.csv', index=False)

In [20]:
barometer = df

In [22]:
barometer

,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
1044,V37547,Lleida,2020-09-24,Swine_influenza_virus,0,Fluid,PCR,Spain,Meat,2,Missing
1045,V37548,Lleida,2020-09-24,Swine_influenza_virus,0,Fluid,PCR,Spain,Meat,2,Missing
1046,V37549,Lleida,2020-09-24,Swine_influenza_virus,0,Fluid,PCR,Spain,Meat,2,Missing
1047,V37550,Lleida,2020-09-24,Swine_influenza_virus,0,Fluid,PCR,Spain,Meat,2,Missing
1048,V37551,Lleida,2020-09-24,Swine_influenza_virus,0,Fluid,PCR,Spain,Meat,2,Missing
...,...,...,...,...,...,...,...,...,...,...,...
1507,V46449,Missing,2022-12-15,Swine_influenza_virus,0,Tissue,PCR,Spain,Meat,2,Missing
1508,V46466,Missing,2022-12-15,Swine_influenza_virus,0,Tissue,PCR,Spain,Meat,2,Missing
1509,V47845,Missing,2022-12-23,Swine_influenza_virus,0,Fluid,PCR,Spain,Meat,2,Missing
1510,V47846,Missing,2022-12-23,Swine_influenza_virus,0,Fluid,PCR,Spain,Meat,2,Missing


### Step 02: Create an RDF graph and namespaces.

In [23]:
g = rdflib.Graph()
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
g.bind('xsd', xsd)

# Define your custom namespace for your ontology's properties
LHO = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/LHO#")
g.bind('LHO', LHO)

SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
g.bind('skos', SKOS)

decide=  Namespace("http://www.purl.org/decide#")
g.bind('decide', decide)
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
g.bind('ncit', ncit)
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")
g.bind('agrovoc', agrovoc)

###  Step 03: Iterate over the Panda DataFrame and map to ontology properties:

In [24]:
#For Pig Data frame RDF conversion
for index, row in barometer.iterrows():
    SampleNumber = row["SampleNumber"]
    Date = row["Date"]
    Pathogen = row["Pathogen"]
    Breed = row["Breed"]
    Country = row["Country"]
    Province = row["Province"]
    SampleType = row["SampleType"]
    SampleResult = row["Result"]
    DiagnosticTest = row["DiagnosticTest"]
    PigStage= row["PigStage"]

    
    # Create a unique URI for each sample based on the row index
    PigSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab2PigSample_{index}")
    
    # Add sample type assertion
    g.add((PigSample_uri, RDF.type, LHO.PigSample))
    description = "An individual representing a sample from a Pig and Piglets."
    g.add((PigSample_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "Pathogen" in row and row["Pathogen"]:
        Pathogen = row["Pathogen"]
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing Pig pathogen."
        g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Breed" in row and row["Breed"]:
        Breed = row["Breed"]
        Breed_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Breed}")
        g.add((PigSample_uri, LHO.hasBreed, Breed_uri))
        g.add((Breed_uri, RDF.type, LHO.Breed))
        description = "An individual representing a Piglet breed which is Meat."
        g.add((Breed_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "DiagnosticTest" in row and row["DiagnosticTest"]:
        DiagnosticTest= row["DiagnosticTest"]
        DiagnosticTest_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{DiagnosticTest}")
        g.add((PigSample_uri, LHO.hasDiagnosticTest, DiagnosticTest_uri))
        g.add((DiagnosticTest_uri, RDF.type, LHO.DiagnosticTest))
    if DiagnosticTest == "PCR":
        description = "An individual representing DNA/RNA amplification for rapid pathogen detection."
    elif SampleType == "Culture":
        description = "An individual representing a Growing live microorganisms for identification."
        g.add((DiagnosticTest_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Country" in row and row["Country"]:
        Country = row["Country"].strip()  # Use strip() to remove leading and trailing spaces
        # Encode the Country value to create a valid URI
        Country = urllib.parse.quote(Country)
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Country}")
    
        # Map to the specific class URI in your ontology and add the label
        g.add((PigSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Use the specific class URI
        g.add((Country_uri, RDFS.label, Literal("Spain")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Province" in row and row["Province"]:
        Province = row["Province"].strip()
        Province = urllib.parse.quote(Province)
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Province}")
        g.add((PigSample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "PigStage" in row and row["PigStage"]:
        Stage = row["PigStage"]
        Stage_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Stage}")
        g.add((PigSample_uri, LHO.hasProductionStages, Stage_uri))
        g.add((Stage_uri, RDF.type, LHO.PigProductionStage))
        description = "An individual representing Production stage ."
        g.add((Stage_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "SampleType" in row and row["SampleType"]:
        SampleType = row["SampleType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        # Add statements for SampleType
        g.add((PigSample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, URIRef("http://www.purl.org/decide#SampleType")))
    if SampleType == "Autopsy":
        description = "An individual representing a cattle sample obtained through autopsy."
    elif SampleType == "BAL":
        description = "An individual representing a cattle sample obtained through bronchoalveolar lavage (BAL)."
    elif SampleType == "SWAB":
        description = "An individual representing a cattle sample obtained through swabbing."
    elif SampleType == "Missing":
        description = "An individual representing a missing or unspecified cattle sample type."
        g.add((SampleType_uri, RDFS.comment, Literal(description, lang="en")))
           
    if "Result" in row and row["Result"]:
        SampleResult = row["Result"]
        # Create a unique URI for the Result based on the value
        Result_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleResult}")
        g.add((PigSample_uri, LHO.hasResult, Result_uri))
        g.add((Result_uri, RDF.type, decide.SampleResult))
    if SampleResult == "1":
        description = "An individual representing a Positive test result"
    elif SampleResult == "0":
        description = "An individual representing a negative test result"
    elif SampleResult == "missing":
        description = "An individual representing an unknown or missing test result."
    g.add((Result_uri, RDFS.comment, Literal(description, lang="en")))


    if "Date" in row and not pd.isnull(row["Date"]):  # Check for NaN or None values
        Date = str(row["Date"]).strip()
    
    # Split the datetime string and take the date part
        Date = Date.split()[0]
    
    # Create a unique URI for the Date based on the value (use the Date directly)
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
    # Add Date assertion
        g.add((PigSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
    

# Serialize the RDF graph to a file
rdf_output_file = "output/PigLab2.rdf"
g.serialize(rdf_output_file, format="xml")

# Serialize the RDF graph to Turtle format and print it too see RDF conversion
turtle_data = g.serialize(format="turtle")
print(turtle_data)

@prefix LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#> .
@prefix decide: <http://www.purl.org/decide#> .
@prefix ncit: <http://purl.obolibrary.org/obo/NCIT_C25464> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

LHO:Lab2PigSample_1044 a LHO:PigSample ;
    LHO:hasBreed LHO:Meat ;
    LHO:hasCountry LHO:Spain ;
    LHO:hasDate LHO:2020-09-24 ;
    LHO:hasDiagnosticTest LHO:PCR ;
    LHO:hasPathogen LHO:Swine_influenza_virus ;
    LHO:hasProductionStages LHO:Missing ;
    LHO:hasProvince LHO:Lleida ;
    LHO:hasResult LHO:0 ;
    LHO:hasSampleType LHO:Fluid ;
    rdfs:comment "An individual representing a sample from a Pig and Piglets."@en .

LHO:Lab2PigSample_1045 a LHO:PigSample ;
    LHO:hasBreed LHO:Meat ;
    LHO:hasCountry LHO:Spain ;
    LHO:hasDate LHO:2020-09-24 ;
    LHO:hasDiagnosticTest LHO:PCR ;
    LHO:hasPathogen LHO:Swine_influenza_virus ;
    LHO:hasProductionStages LHO:Missing ;
    LHO:hasProvince LHO:Lleida ;
    LHO:hasResult LHO:0 ;
    LHO:ha

### Step 4: For Knowledge Graph,Load the RDF data and ontology into a Panda DataFrame: 

In [25]:
# Parse the ontology file in OWL format and add it to the graph
path_to_ontology = "Ontology/LivestockHealthOntology.owl"
g.parse(path_to_ontology, format="xml")

<Graph identifier=N7c140a47b71240adbcc4ef996151c9ce (<class 'rdflib.graph.Graph'>)>

### Step 05: Query the data from updated ontology 

In [26]:

query = """
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT
  (strafter(str(?Sample), "#") AS ?SampleName)
  (strafter(str(?Pathogen), "#") AS ?PathogenName)
  (strafter(str(?Breed), "#") AS ?BreedName)
  (strafter(str(?SampleType), "#") AS ?SampleTypeName)
  (strafter(str(?DiagnosticTest), "#") AS ?DiagnosticTestName)
  (strafter(str(?PigProductionStages), "#") AS ?ProductionStage)
  (strafter(str(?Province), "#") AS ?ProvinceName)
  (strafter(str(?Country), "#") AS ?CountryName)
  (strafter(str(?SampleResult), "#") AS ?SampleResultName)

WHERE {
 {
  ?Sample rdf:type LHO:PigSample .
  ?Sample LHO:hasPathogen ?Pathogen .
  ?Sample LHO:hasBreed ?Breed .
  ?Sample LHO:hasSampleType ?SampleType .
  ?Sample LHO:hasProvince ?Province .
  ?Sample LHO:hasCountry ?Country .
  ?Sample LHO:hasProductionStages ?PigProductionStages .
  ?Sample LHO:hasResult ?SampleResult .
  FILTER (?SampleResult = LHO:1)
  ?Sample LHO:hasDiagnosticTest ?DiagnosticTest .
  }
}
"""


# Execute the query and retrieve the results
results = g.query(query)

# Convert the results to a Pandas dataframe
data = []
for row in results:
    data.append(list(row))

df = pd.DataFrame(data, columns=[
    "Sample", "Pathogen", "Breed", "SampleType", 
    "DiagnosticTest","PigProductionStage","Province", "Country", "SampleResult"
])

# Display the dataframe
df


,Sample,Pathogen,Breed,SampleType,DiagnosticTest,PigProductionStage,Province,Country,SampleResult
0,Lab2PigSample_1056,Swine_influenza_virus,Meat,Fluid,PCR,Missing,Missing,Spain,1
1,Lab2PigSample_1063,Swine_influenza_virus,Meat,Tissue,PCR,Missing,Lleida,Spain,1
2,Lab2PigSample_1071,Swine_influenza_virus,Meat,Fluid,PCR,Missing,Missing,Spain,1
